## import library and load Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pyswarms as ps
from deap import base, creator, tools, algorithms

In [9]:
train_df = pd.read_csv('loan_train.csv')
test_df = pd.read_csv('loan_test.csv')

print(f"Row Number Train: {train_df.shape[0]}")
print(f"Row Number Test: {test_df.shape[0]}")
print(f"Columns :\n{train_df.columns.tolist()}")

Row Number Train: 614
Row Number Test: 367
Columns :
['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount', 'Term', 'Credit_History', 'Area', 'Status']


## Data Preprocessing